Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам в словаре применяется только одна операция - удаление символа (1-n). Чтобы найти исправление из слова тоже удаляются символы и сравниваются с теми, что хранятся в словаре. Оцените качество полученного алгоритма теми же тремя метриками.

https://medium.com/@wolfgarbe/1000x-faster-spelling-correction-algorithm-2012-8701fcd87a5f

Добавьте к полученному алгоритму исправления триграммную модель и проверьте, улучшает ли она качество.

In [1]:
from collections import Counter
from nltk import sent_tokenize
from string import punctuation
import gzip
import csv
import pandas as pd
import re

In [2]:
data = pd.read_csv('lenta-ru-news.csv')

In [3]:
texts = data['text']

In [4]:
punctuation +=  "«»—…“”"
punct = set(punctuation )

In [5]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token
                in tokens_1 if (set(token) - punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token
                in tokens_2 if (set(token) - punct)]
    
    return zip(tokens_1, tokens_2)

In [6]:
def normalize(text):
    normalized = [(word.strip(punctuation)) for word 
                  in text.lower().split()
                  if word.isalpha() 
                  and re.fullmatch(r'[а-я]+', word.strip(punctuation))]
    return [word for word in normalized if word]

In [7]:
corpus = []
for text in texts[:8000]:
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

In [8]:
vocab = set()

for sent in corpus:
    vocab.update(sent)

In [9]:
def n_deletes_helper(word, min_len=0):
    result = set()
    # taking word length into account
    if len(word) > min_len:
        for i in range(len(word)):
            result.add(word[:i] + word[i+1:])
    else:
        result.add(word)
    return result

In [10]:
def n_deletes(word, edit_distance=2, min_len=2):
    forms = {word}
    while edit_distance > 0:
        edit_distance -= 1
        new_forms = set()
        for form in forms:
            new_forms.update(n_deletes_helper(form, min_len))
        forms = new_forms
        
    return forms

In [14]:
#n_deletes('word', edit_distance=2)

In [36]:
sym_vocab = {}
for word in vocab:
    forms = n_deletes(word)
    for form in forms:
        if form not in sym_vocab.keys():
            sym_vocab[form] = word
        else:
            if isinstance(sym_vocab[form], set):
                sym_vocab[form].add(word)
            else:
                sym_vocab[form] = {sym_vocab[form]}
                sym_vocab[form].add(word)

```
There are four different comparison pair types:
dictionary entry==input entry,
delete(dictionary entry,p1)==input entry
dictionary entry==delete(input entry,p2)
delete(dictionary entry,p1)==delete(input entry,p2)
```